In [2]:
#imports and authentications

import spotipy
import pandas as pd
import datetime
import time
import os
from json.decoder import JSONDecodeError
import numpy as np
import pandas as pd

# music api imports
import pygn
import traceback
import spotipy
import spotipy.util as util

import requests
import re
from time import sleep
import pickle

import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

scope = 'user-library-read'
username = "karishma.gulati"
client_id="afe7c584ab704e5487b31b52a9d0009c"
client_secret="fa08966efd794dbb88a484e9a54b11e1"
redirect_uri="http://localhost:8888/"

# token = util.prompt_for_user_token(<INSERT SPOTIFY USERNAME>,
#                                   client_id=<INSERT SPOTIFY CLIENT ID>,
#                                   client_secret=<INSERT SPOTIFY CLIENT SECRET>,
#                                   redirect_uri=<INSERT REDIRECT URI>)

try:
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
except (AttributeError, JSONDecodeError):
    os.remove(f".cache-{username}")
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

spotify = spotipy.Spotify(auth=token)

In [16]:
#defining functions

# get categories
def get_categories():
    category_ids = []
    for i in spotify.categories(limit = 50)['categories']['items']:
        category_ids.append(i.get('id'))
    return category_ids

# get playlists from list of categories
def get_playlists(categories):
    playlist_ids = []
    for i in categories:
        for j in spotify.category_playlists(i, limit = 50)['playlists']['items']:
            playlist_ids.append(j.get('id'))
    return playlist_ids

# get song ids from list of playlist ids
def get_songs(playlists):
    song_ids = []
    count = 0
    for i in playlists:
        try:
            for j in spotify.user_playlist('spotify', i)['tracks']['items']:
                song_ids.append([j['track']['id'], j['track']['name'], j['track']['artists'][0]['name']])
        except:
            pass
        time.sleep(.1)
    return song_ids

def chunks(l, n):
    n = max(1, n)
    return [l[i:i+n] for i in range(0, len(l), n)]

def complete_audio_features(songs_ids):
    features_list = []
    for j in chunks(list(songs_ids),50):
        features_add = spotify.audio_features(tracks=j)
        features_list.extend(features_add)
    return features_list

def make_audio_features_df(audio_features):
    audio_features_df = pd.DataFrame(columns = audio_features[0])
    for i in range(0,len(audio_features)):
        audio_features_df = audio_features_df.append(audio_features[i], ignore_index=True)
    return audio_features_df

In [17]:
# Getting Song Information for these song IDs

In [18]:
print("The spotify token has been fetched!\n")
categories = get_categories()
playlists = get_playlists(categories)
songs = get_songs(playlists)
songs_df = pd.DataFrame(songs)
songs_df.head(10)

The spotify token has been fetched!



,0,1,2
0,3YU6vJbjYUG0tiJyXf9x5V,Him & I (with Halsey),G-Eazy
1,1bhUWB0zJMIKr9yVPrkEuI,Perfect Duet (Ed Sheeran & Beyoncé),Ed Sheeran
2,0OI7AFifLSoGzpb8bdBLLV,"Home (with Machine Gun Kelly, X Ambassadors & ...",Machine Gun Kelly
3,2amzBJRBPOGszBem4FedfE,Let You Down,NF
4,42CeaId2XNlxugDvyqHfDf,Candy Paint,Post Malone
5,2xmrfQpmS2iJExTlklLoAL,I Miss You (feat. Julia Michaels),Clean Bandit
6,0tBbt8CrmxbjRP0pueQkyU,Wolves,Selena Gomez
7,7eFmN6wnsb7WowRKAqRFfs,Wait,Maroon 5
8,2z4pcBLQXF2BXKFvd0BuB6,Tip Toe (feat. French Montana),Jason Derulo
9,3al2hpm92xE0pBalqWQHdD,Betrayed,Lil Xan


In [19]:
songs_df.columns = ['id', 'name', 'artist']
songs_df = songs_df.drop_duplicates()
print("Number of unique songs: ", len(songs_df.index))

Number of unique songs:  36016


In [21]:
audio_features = complete_audio_features(songs_df['id'])
audio_features_df = make_audio_features_df(audio_features)
audio_features_df = audio_features_df.drop_duplicates()
print("Number of unique songs post receiving audio features: ", len(audio_features_df.index))

Number of unique songs post receiving audio features:  35862


In [22]:
songs_final_df = songs_df.merge(audio_features_df, how='inner', on='id')
del songs_final_df['track_href']
del songs_final_df['analysis_url']
del songs_final_df['uri']
del songs_final_df['type']

songs_final_df.head(10)

,id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,3YU6vJbjYUG0tiJyXf9x5V,Him & I (with Halsey),G-Eazy,0.589,0.731,2,-6.343,1,0.0868,0.05340,0,0.3080,0.191,87.908,268867,4
1,1bhUWB0zJMIKr9yVPrkEuI,Perfect Duet (Ed Sheeran & Beyoncé),Ed Sheeran,0.587,0.299,8,-7.365,1,0.0263,0.77900,0,0.1230,0.356,94.992,259550,3
2,0OI7AFifLSoGzpb8bdBLLV,"Home (with Machine Gun Kelly, X Ambassadors & ...",Machine Gun Kelly,0.653,0.718,3,-5.232,0,0.2130,0.00413,0,0.0537,0.216,82.034,202805,4
3,2amzBJRBPOGszBem4FedfE,Let You Down,NF,0.661,0.715,5,-5.651,0,0.1190,0.31600,0,0.1780,0.411,148.027,212120,4
4,42CeaId2XNlxugDvyqHfDf,Candy Paint,Post Malone,0.765,0.688,4,-5.566,1,0.0841,0.63300,0,0.0734,0.434,90.013,229573,4
5,2xmrfQpmS2iJExTlklLoAL,I Miss You (feat. Julia Michaels),Clean Bandit,0.638,0.658,3,-6.318,1,0.0456,0.24500,3.77e-06,0.0919,0.330,105.076,205748,4
6,0tBbt8CrmxbjRP0pueQkyU,Wolves,Selena Gomez,0.720,0.807,11,-4.590,0,0.0432,0.12900,0,0.1830,0.305,124.946,197993,4
7,7eFmN6wnsb7WowRKAqRFfs,Wait,Maroon 5,0.648,0.608,8,-5.160,1,0.0587,0.08350,0,0.1050,0.488,126.120,190643,4
8,2z4pcBLQXF2BXKFvd0BuB6,Tip Toe (feat. French Montana),Jason Derulo,0.845,0.709,10,-4.547,0,0.0714,0.02330,0,0.0940,0.620,98.062,187521,4
9,3al2hpm92xE0pBalqWQHdD,Betrayed,Lil Xan,0.827,0.419,10,-10.329,0,0.1120,0.41300,0,0.1150,0.227,119.974,187250,4


In [23]:
print("Final song list has songs: ", len(songs_final_df.index))

Final song list has songs:  35862


In [ ]:
songs_final_df.to_csv("all_songs.csv", index=False)

In [ ]:
#Merge all songs with features from Gracenote

In [4]:
# timing function
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print(f.__name__, 'took', end - start, 'seconds')
        return result
    return f_timer

In [5]:
clientid='1143049347-DB6E8C78665AF7AE447BE4903B6F2BF5'
userid = pygn.register(clientid)

In [6]:
songs = pd.read_csv('all_songs.csv')
songs.head(10)

,Unnamed: 0,id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,7eFmN6wnsb7WowRKAqRFfs,Wait,Maroon 5,0.648,0.608,8,-5.160,1,0.0587,0.0835,0.000000,0.1050,0.488,126.120,190643,4
1,1,42CeaId2XNlxugDvyqHfDf,Candy Paint,Post Malone,0.765,0.688,4,-5.566,1,0.0841,0.6330,0.000000,0.0734,0.434,90.013,229573,4
2,2,0tBbt8CrmxbjRP0pueQkyU,Wolves,Selena Gomez,0.720,0.807,11,-4.590,0,0.0432,0.1290,0.000000,0.1830,0.305,124.946,197993,4
3,3,3whrwq4DtvucphBPUogRuJ,Havana - Remix,Camila Cabello,0.751,0.579,2,-4.036,1,0.0321,0.3980,0.000023,0.1330,0.349,105.031,199095,4
4,4,2z4pcBLQXF2BXKFvd0BuB6,Tip Toe (feat. French Montana),Jason Derulo,0.845,0.709,10,-4.547,0,0.0714,0.0233,0.000000,0.0940,0.620,98.062,187521,4
5,5,2xmrfQpmS2iJExTlklLoAL,I Miss You (feat. Julia Michaels),Clean Bandit,0.638,0.658,3,-6.318,1,0.0456,0.2450,0.000004,0.0919,0.330,105.076,205748,4
6,6,2amzBJRBPOGszBem4FedfE,Let You Down,NF,0.661,0.715,5,-5.651,0,0.1190,0.3160,0.000000,0.1780,0.411,148.027,212120,4
7,7,7iDa6hUg2VgEL1o1HjmfBn,Meant to Be (feat. Florida Georgia Line),Bebe Rexha,0.643,0.783,10,-6.458,1,0.0856,0.0470,0.000000,0.0830,0.579,154.084,163870,4
8,8,4utgozKpeqZA18lFCcg70Q,Get It Right (feat. MØ),Diplo,0.456,0.833,0,-4.242,1,0.0964,0.1080,0.000017,0.0592,0.261,159.808,172500,4
9,9,5Gu0PDLN4YJeW75PpBSg9p,"Let Me Go (with Alesso, Florida Georgia Line &...",Hailee Steinfeld,0.667,0.726,8,-4.172,1,0.0540,0.0320,0.000000,0.0745,0.770,103.001,174800,4


In [8]:
# Get track info from gracenote
# NOTE: Gracenote API has built-in fuzzy matching to artist and track.
from collections import defaultdict

def get_gn_multiple(search, dictionary, item):
    '''
    Helper function to get multiple items within Gracenote record
    '''
    for i in search[item].items():
        dictionary[item + '_' + i[0]] = i[1]['TEXT']

def get_gn(artist, track):
    '''
    Gets artist and track information from Gracenote
    '''
    gn_dict = defaultdict(list)
    gn_info = pygn.search(clientid, userid, artist=artist, track=track)

    gn_dict['gnid'] = gn_info['track_gnid']

    # artist specific info
    for a in ['artist_origin', 'artist_type', 'artist_era']:
        get_gn_multiple(gn_info, gn_dict, a)
    # track specific info
    for s in ['genre', 'mood', 'tempo']: # can potentially drop 'tempo' since Spotify has already captured this
        get_gn_multiple(gn_info, gn_dict, s)      
    return dict(gn_dict)

@timefunc
def getDetailsFromGracenote(df):
    d = {}
    for index, row in df.iterrows():
        try:
            d[row['id']] = get_gn(row['artist'], row['name'])
        except:
            pass

    gn_df = pd.DataFrame(d).transpose().reset_index()
    gn_df = gn_df.rename(columns = {'index' : 'id'})
    return gn_df

In [9]:
final_gn_df = getDetailsFromGracenote(songs)
final_gn_df

getDetailsFromGracenote took 4312.954648256302 seconds


,id,artist_era_1,artist_era_2,artist_origin_1,artist_origin_2,artist_origin_3,artist_origin_4,artist_type_1,artist_type_2,genre_1,genre_2,genre_3,gnid,mood_1,mood_2,tempo_1,tempo_2,tempo_3
0,0007hNErjiadpYWXC0Tnof,Pre-20th C.,NaN,Western Europe,Germany,NaN,NaN,Male,Male,Classical,Baroque Era,Baroque Instrumental,330313826-DEA027579F5EAFF32DEC1321CBE52D3B,Tender,Refined / Mannered,Medium Tempo,Medium Slow,50s
1,000EWWBkYaREzsBplYjUag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Urban,Western Hip-Hop/Rap,General Hip-Hop/Rap,760962305-F8D310AFEC1D0BE77D9C9E619207578D,Sensual,Dreamy Pulse,Medium Tempo,Medium,70s
2,006AVH7fq061voGXkUiII4,2010s,Early 2010s,Western Europe,Ireland,NaN,Dublin,Male,Male,Pop,Western Pop,Adult Alternative Pop,558013277-85F8BA64807BEFE1D20F8AD0FA6CD15D,Empowering,Strong / Stable,Medium Tempo,Medium Fast,90s
3,006UajgUjRpX0RfnoxphRJ,2000s,NaN,Western Europe,Netherlands,NaN,NaN,Male,Male,Electronica,Trance,Trance,754669605-2E9F9077545C93D06E4DC3E5CD63949C,NaN,NaN,NaN,NaN,NaN
4,00OzUEgsPIqPK1LGaQBHMk,2010s,Mid 2010s,North America,United States,New York,New York City,Male,Male,Electronica,House,Deep House,626560293-B6E726AC8077F33FC84CC79EEDDD389B,NaN,NaN,NaN,NaN,NaN
5,00QIoJwUK7ElUbHYIPeqSU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rock,Mainstream Rock,General Mainstream Rock,623044040-E374CD9B749C24B71B6C5839BA6AB3E5,Tender,Light Groovy,Medium Tempo,Medium,70s
6,00b0Eq19oRIOZYIIM1b0KJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pop,Western Pop,Pop Vocal,755307095-01457C77894C0375AD02F85C1F66F6D9,Empowering,Dramatic Emotion,Medium Tempo,Medium Slow,50s
7,00kkWwGsR9HblTUHb3BmdX,2010s,NaN,North America,United States,"Georgia, US",NaN,Male,Male,Other,Religious,Contemporary Gospel,715051221-E67194B6E2F3F60A46DA464DA4F6913B,Sensual,Intimate,Slow Tempo,Slow,40s
8,00o4GuYe2ZbyEtrjqiik58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pop,Western Pop,Pop Vocal,544878737-F91B15BA8A72B7A29924F14BC32563A1,NaN,NaN,NaN,NaN,NaN
9,016jMrTAKPT7Xu6X19xdTM,2010s,Mid 2010s,Australia/Oceania,Australia,Victoria,Melbourne,Female,Female,Urban,Classic R&B/Soul,R&B/Soul,511416905-BF927B12F74DC255F07226642A74926A,Cool,Wary / Defiant,Medium Tempo,Medium,70s


In [ ]:
songs_final_df = songs.merge(final_gn_df, how='inner', on='id')
print("Final song list without eliminating duplicates: ", len(songs_final_df.index))

songs_final_df = songs_final_df.drop_duplicates()
print("Final song list after eliminating duplicates: ", len(songs_final_df.index))

#songs_final_df.to_csv("all_songs_with_labels.csv", index=False)

In [19]:
all_songs = pd.read_csv("all_songs_with_labels.csv")
all_songs = all_songs[pd.notnull(all_songs['mood_1'])]
all_songs.mood_1.unique()
print("Songs without NaN label values", len(all_songs.index))

all_songs = all_songs[all_songs.mood_1 != 'Other']
print("Songs without Other label values", len(all_songs.index))
all_songs.mood_1.unique()

#Excited = happy
#Sensual = Relaxed
#Rowdy = Angry
#Cool = Relaxed
#Defiant = Angry
#Energizing = happy
#Urgent = Angry
#Sophisticated = Relaxed
#Yearning = Sad
#Melancholy = Sad
#Brooding = Sad
#Lively = happy
#Empowering = happy
#Aggressive = angry
#Upbeat = Happy
#Somber = Sad
#Peaceful = Relaxed
#Gritty = Sad
#Fiery = Angry
#Easygoing = Relaxed
#Romantic = happy
#Sentimental = sad
#Stirring = sad
#Tender = relaxed
#Serious = sad

all_songs.mood_1[all_songs.mood_1 == 'Excited'] = 'Happy' 
all_songs.mood_1[all_songs.mood_1 == 'Sensual'] = 'Relaxed'
all_songs.mood_1[all_songs.mood_1 == 'Rowdy'] = 'Angry'
all_songs.mood_1[all_songs.mood_1 == 'Cool'] = 'Relaxed'
all_songs.mood_1[all_songs.mood_1 == 'Defiant'] = 'Angry'
all_songs.mood_1[all_songs.mood_1 == 'Energizing'] = 'Happy'
all_songs.mood_1[all_songs.mood_1 == 'Urgent'] = 'Angry'
all_songs.mood_1[all_songs.mood_1 == 'Sophisticated'] = 'Relaxed'
all_songs.mood_1[all_songs.mood_1 == 'Yearning'] = 'Sad'
all_songs.mood_1[all_songs.mood_1 == 'Melancholy'] = 'Sad'
all_songs.mood_1[all_songs.mood_1 == 'Brooding'] = 'Sad'
all_songs.mood_1[all_songs.mood_1 == 'Lively'] = 'Happy'
all_songs.mood_1[all_songs.mood_1 == 'Empowering'] = 'Happy'
all_songs.mood_1[all_songs.mood_1 == 'Aggressive'] = 'Angry'
all_songs.mood_1[all_songs.mood_1 == 'Upbeat'] = 'Happy'
all_songs.mood_1[all_songs.mood_1 == 'Somber'] = 'Sad'
all_songs.mood_1[all_songs.mood_1 == 'Peaceful'] = 'Relaxed'
all_songs.mood_1[all_songs.mood_1 == 'Gritty'] = 'Sad'
all_songs.mood_1[all_songs.mood_1 == 'Fiery'] = 'Angry'
all_songs.mood_1[all_songs.mood_1 == 'Easygoing'] = 'Relaxed'
all_songs.mood_1[all_songs.mood_1 == 'Romantic'] = 'Happy'
all_songs.mood_1[all_songs.mood_1 == 'Sentimental'] = 'Sad'
all_songs.mood_1[all_songs.mood_1 == 'Stirring'] = 'Sad'
all_songs.mood_1[all_songs.mood_1 == 'Tender'] = 'Relaxed'
all_songs.mood_1[all_songs.mood_1 == 'Serious'] = 'Sad'

all_songs.mood_1.unique()

Songs without NaN label values 4510
Songs without Other label values 4446


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/

array(['Happy', 'Relaxed', 'Angry', 'Sad'], dtype=object)

In [21]:
#all_songs.to_csv("all_songs_with_modified_labels.csv", index=False)